## Reusable methods to be included in other Notebook files

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import requests
import json
import time
from IPython.display import Markdown
import os.path
import random
import hashlib

In [2]:
# colors with a good contrast to white and among each other
# I use black for BTC
colors = ['red', 'orange', 'green', 'purple', 'brown', 'pink', 'olive', 'cyan', 'grey'] # add more colors if needed


In [3]:
from IPython.display import IFrame
def getTradingViewWidget(ticker="Coinbase:BTCUSD"):
    chart_id = "xxxxx"
    # Unfortunately that doesn't work 
    # https://www.tradingview.com/charting-library-docs/latest/api/interfaces/Charting_Library.ChartingLibraryWidgetOptions/#timeframe
    # &timeframe={{from:1640995200,to:1643673600}}
    # &timeframe=%7Bfrom%3A1640995200%2Cto%3A1643673600%7D
    # As well as the inverval and probably other parameters
    src = f'''https://s.tradingview.com/widgetembed/?frameElementId=tradingview_{chart_id}
    &symbol={ticker}&interval=D&hidesidetoolbar=1&symboledit=1&saveimage=0&toolbarbg=f1f3f6&studies=%5B%5D&style=1&timezone=UTC&timeframe=%7Bfrom%3A1640995200%2Cto%3A1643673600%7D"
    '''
    return IFrame(src, '100%', '600px')


In [4]:
#getTradingViewWidget()

In [5]:
def showPlainPriceChart(df, strike_prices):
    fig = go.Figure()
    min_price = 0
    max_price = 0
    for i, strike_price in enumerate(strike_prices):
        df_strike = df[f'option_{strike_price}_usd']
        min_price = min(min_price, df_strike.min())
        max_price = max(max_price, df_strike.max())
        fig.add_trace(go.Scatter(x=df['timestamp'], 
                                 y=df_strike, 
                                 name=f'Option {strike_price}', 
                                 yaxis='y', 
                                 line=dict(color=colors[i])))
        
    fig.add_trace(go.Scatter(x=df['timestamp'], 
                             y=df['btc_usd'], 
                             name='BTC', 
                             yaxis='y', 
                             line=dict(color="black")))
    
    range = [min(min_price, df['btc_usd'].min()), max(max_price, df['btc_usd'].max())]
    
    fig.update_layout(width=1300, height=800)
    fig.update_layout(title="Plain prices in USD", yaxis=dict(title='Prices in USD', range=range))
    fig.update_traces(connectgaps=False)
    fig.show()

In [6]:
class Config:
    def __init__(self, config_dict):
        self.__dict__.update(config_dict)

In [7]:
def getBTCCallInstrumentName(expiration, strike):
    instrument = 'BTC-' + expiration + '-' + str(strike) + '-C'
    return instrument

In [8]:
def getInstrumentJson(instrumentName):
    fileName = "option_data/" + instrumentName + ".json"
    if os.path.isfile(fileName):
        display(f"{fileName} exist already.")
        df = pd.read_json(fileName)
        return pd.json_normalize(df.trades)
    else:
        print(f"Downloading to {fileName}.")
        url = f'https://history.deribit.com/api/v2/public/get_last_trades_by_instrument?instrument_name={instrumentName}&count=10000&include_old=true'

        response = requests.get(url)

        # Parse the JSON response
        result = json.loads(response.text)['result']
        with open(fileName, 'w') as f:
            json.dump(result, f)
        return pd.json_normalize(result['trades'])
    df = df.drop_duplicates(subset=['timestamp'])
    return df

In [9]:
def getBTCdataframe():
    btc_ohlc = pd.read_json('bitstamp-btc-ohlc.json')
    btc_ohlc['timestamp'] = pd.to_datetime(btc_ohlc['timestamp']).dt.date
    btc_ohlc = btc_ohlc.loc[:, ['timestamp', 'close']].copy()
    return btc_ohlc

In [10]:
def getOptionDataframe(expiration, strike):
    df = getInstrumentJson(getBTCCallInstrumentName(expiration, strike))
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms').dt.date
    df = df.loc[:, ['timestamp', 'price']].copy()
    df = df.drop_duplicates(subset=['timestamp'])
    return df

In [11]:
def getMergedDataframe(config):
    merged_df = getOptionDataframe(config.expiration, config.strikes[0])
    merged_df.rename({'price': f'option_{config.strikes[0]}_usd'}, axis=1, inplace=True)

    for i, strike_price in enumerate(config.strikes[1:]):
        dfStrike = getOptionDataframe(config.expiration, config.strikes[i])
        dfStrike.rename({'price': f'option_{strike_price}_usd'}, axis=1, inplace=True)
        merged_df = pd.merge(merged_df, dfStrike, on='timestamp', how='outer')
        
    btc_ohlc = getBTCdataframe()
    # Join options with BTC prices for all option timestamp that are contained within the btc_ohlc
    # but not the ones that are contained within btc_ohlc but don't exist in the merged_df of the options.
    merged_df = pd.merge(btc_ohlc, merged_df, on='timestamp', how='right')
    merged_df = merged_df.drop_duplicates(subset=['timestamp'])
    merged_df.rename({'close': 'btc_usd'}, axis=1, inplace=True)
    
    for i, strike_price in enumerate(config.strikes):
        merged_df[f'option_{strike_price}_usd'] = merged_df[f'option_{strike_price}_usd'] * merged_df['btc_usd']
    
    return merged_df


In [12]:
def showOffsettedPriceChart(df, strike_prices):
    fig = go.Figure()

    min_price = df['btc_usd'].min()
    max_price = df['btc_usd'].max()
    for i, strike_price in enumerate(strike_prices):
        df_strike = df[f'option_{strike_price}_offsetted_usd']
        min_price = min(min_price, df_strike.min())
        max_price = max(max_price, df_strike.max())
        fig.add_trace(go.Scatter(x=df['timestamp'], 
                                 y=df_strike, 
                                 name=f'Option {strike_price}', 
                                 yaxis='y', 
                                 line=dict(color=colors[i])))
        
    fig.add_trace(go.Scatter(x=merged_df['timestamp'],
                             y=merged_df['btc_usd'],
                             name='BTC', 
                             yaxis='y',
                             line=dict(color="black")))

    range = [min(min_price, df['btc_usd'].min()), max(max_price, df['btc_usd'].max())]

    fig.update_layout(width=1300, height=800)

    fig.update_layout(title="Option prices moved to start at the same point as BTC to visualize the delta", yaxis=dict(title='Prices in USD', range=range))

    fig.show()

In [13]:
def shiftLinesToTheStartOfBTC(df, strike_prices):
    firstBTCprice = df.loc[df.index[-1], 'btc_usd']
    
    for i, strike_price in enumerate(config.strikes):
        firstOptionPrice = df.loc[df.index[-1], f'option_{strike_price}_usd']
        strike_offset = firstBTCprice - firstOptionPrice
        df[f'option_{strike_price}_offsetted_usd'] = df[f'option_{strike_price}_usd'] + strike_offset

#    return df


In [14]:
def getPriceChanges(df, strike_prices):
    dfNew = pd.DataFrame(df['timestamp'])
    dfNew = pd.concat([dfNew, df['btc_usd']], axis=1)

    for i, strike_price in enumerate(strike_prices):
        strike_col = df[f'option_{strike_price}_usd']
        dfNew[f'option_{strike_price}_changed_usd'] = strike_col - strike_col.shift(-1)
        dfNew[f'option_{strike_price}_changed_percent'] = dfNew[f'option_{strike_price}_changed_usd'] / (df[f'option_{strike_price}_usd']/100)
    dfNew['btc_usd_changed_usd'] =  df['btc_usd'] - df['btc_usd'].shift(-1)
    dfNew['btc_usd_changed_percent'] = dfNew['btc_usd_changed_usd'] / (df['btc_usd']/100)
    return dfNew

In [15]:
def plotOptionPricesChangedUSD(df, strikes):
    
    fig = go.Figure()
    
    for i, strike in enumerate(strikes):
        fig.add_trace(go.Scatter(x=df['timestamp'],
                                 y=df[f'option_{strike}_changed_usd'],
                                 name=f'Option {strike}', yaxis='y',
                                 line=dict(color=colors[i % len(colors)])))
    
    fig.add_trace(go.Scatter(x=df['timestamp'],
                             y=df['btc_usd_changed_usd'],
                             name='BTC', yaxis='y',
                             line=dict(color='black')))
    
    fig.add_trace(go.Scatter(x=df['timestamp'],
                             y=df['btc_usd'],
                             name='BTC price in USD on separate axis', yaxis='y2',
                             line=dict(color='gold')))
    
    range = [min([df[f'option_{strike}_changed_usd'].min() for strike in strikes] + [df['btc_usd_changed_usd'].min()]),
             max([df[f'option_{strike}_changed_usd'].max() for strike in strikes] + [df['btc_usd_changed_usd'].max()])]
    
    fig.update_layout(width=1600, height=800)
    fig.update_layout(title="Absolute changes in USD vs BTC price",
                      yaxis=dict(title='Absolute changes in USD', range=range),
                      yaxis2=dict(title='BTC in USD (in gold)',
                                  range=[df['btc_usd'].min(), df['btc_usd'].max()],
                                  overlaying='y', side='right'))
    
    fig.show()

In [16]:
def plotOptionPricesChangedPercent(df, strikes):
    
    fig = go.Figure()
    
    for i, strike in enumerate(strikes):
        fig.add_trace(go.Scatter(x=df['timestamp'],
                                 y=df[f'option_{strike}_changed_percent'],
                                 name=f'% change option {strike}', yaxis='y',
                                 line=dict(color=colors[i % len(colors)])))
    
    fig.add_trace(go.Scatter(x=df['timestamp'],
                             y=df['btc_usd_changed_percent'],
                             name='% change BTC', yaxis='y',
                             line=dict(color='black')))

    fig.add_trace(go.Scatter(x=df['timestamp'],
                             y=df['btc_usd'],
                             name='BTC price in USD on separate axis', yaxis='y2',
                             line=dict(color='gold')))
    '''
    '''
    range = [min([df[f'option_{strike}_changed_percent'].min() for strike in strikes] + [df['btc_usd_changed_percent'].min()]),
             max([df[f'option_{strike}_changed_percent'].max() for strike in strikes] + [df['btc_usd_changed_percent'].max()])]
    
    fig.update_layout(width=1600, height=800)
    fig.update_layout(title="Changes in % vs BTC price",
                      yaxis=dict(title='Changes in %', range=range),
                      yaxis2=dict(title='BTC in USD (in gold)',
                                  range=[df['btc_usd'].min(), df['btc_usd'].max()],
                                  overlaying='y', side='right')
                     )
    
    fig.show()

In [17]:
def plotOptionPricesChangedPercentAll(df, strikes):
    
    fig = go.Figure()
    
    for i, strike in enumerate(strikes):
        fig.add_trace(go.Scatter(x=df['timestamp'],
                                 y=df[f'option_{strike}_percent_all'],
                                 name=f'% change {strike} option', yaxis='y',
                                 line=dict(color=colors[i % len(colors)])))
    
    fig.add_trace(go.Scatter(x=df['timestamp'],
                             y=df['btc_percent_all'],
                             name='% change BTC', yaxis='y',
                             line=dict(color='black')))

    fig.add_trace(go.Scatter(x=df['timestamp'],
                             y=df['btc_usd'],
                             name='BTC price in USD on separate axis', yaxis='y2',
                             line=dict(color='gold')))
    '''
    '''
    range = [min([df[f'option_{strike}_percent_all'].min() for strike in strikes] + [df['btc_percent_all'].min()]),
             max([df[f'option_{strike}_percent_all'].max() for strike in strikes] + [df['btc_percent_all'].max()]) * 1.01]
    
    fig.update_layout(width=1600, height=800)
    fig.update_layout(title="Complete changes in % vs BTC price",
                      yaxis=dict(title='Complete changes in %', range=range),
                      yaxis2=dict(title='BTC in USD (in gold)',
                                  range=[df['btc_usd'].min(), df['btc_usd'].max() * 1.01],
                                  overlaying='y', side='right')
                     )
    
    fig.show()

In [18]:
def calcGains(df, strikes):
    dfNew = pd.DataFrame(df['timestamp'])
    dfNew = pd.concat([dfNew, df['btc_usd']], axis=1)

    dfNew.loc[:, 'btc_percent_all'] = (df['btc_usd'] - df.loc[df.index[-1], 'btc_usd']) / (df.loc[df.index[-1], 'btc_usd'] / 100)

    for i, strike_price in enumerate(strikes):
        option_usd_col_name = f'option_{strike_price}_usd'
        dfNew[option_usd_col_name] = df[option_usd_col_name].copy()
        dfNew[f'option_{strike_price}_percent_all'] = ( 
            df[option_usd_col_name] 
            - 
            df.loc[df[option_usd_col_name].last_valid_index(), option_usd_col_name]
        ) / (
            df.loc[df[option_usd_col_name].last_valid_index(), option_usd_col_name] 
            / 
            100
        )
    return dfNew
